In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from uncertainties import *

from uncertainties import ufloat
from uncertainties.umath import *  # sin(), etc.
import uncertainties.umath
import uncertainties

#definitions of the constants
capacitor_distance=0.006 #m
viscosity_air=1.81e-5 #Ns/m^2
density_oil=875 #kg/m^3
density_air=1.29 #kg/m^3
step_value=0.00005 #m
g=9.81 #m/s^2

In [11]:
SinkSteig_data=pd.read_csv("Milikan_sinkSteig.csv", header=2)
SinkSteig_data

,U/V,U_err/V,t1/s,t1_err/s,t2/s,t2_err/s,scale-steps,scale-steps_err
0,504,1,6.122,0.01,3.281,0.01,3,1.732051
1,512,1,6.634,0.01,2.450,0.01,3,1.732051
2,511,1,4.119,0.01,3.400,0.01,3,1.732051
3,511,1,2.525,0.01,0.862,0.01,3,1.732051
4,523,1,2.881,0.01,2.368,0.01,3,1.732051
5,568,1,8.203,0.01,1.675,0.01,3,1.732051
6,513,1,4.108,0.01,2.883,0.01,3,1.732051
7,515,1,4.536,0.01,3.050,0.01,3,1.732051
8,515,1,5.835,0.01,2.271,0.01,3,1.732051
9,534,1,2.333,0.01,2.204,0.01,3,1.732051


In [20]:
schweben_data=pd.read_csv("Milikan_Schweben.csv")
schweben_data

,U/V,U_err/V,t/s,t_err/s,s/Skalen,s_err
0,225,1,6.070,0.01,5,2.236068
1,138,1,5.281,0.01,3,1.732051
2,113,1,6.591,0.01,3,1.732051
3,175,1,3.573,0.01,3,1.732051
4,56,1,4.193,0.01,3,1.732051


Assume the droplet is $\textbf{floating}$. Hence we have no friction of air. Thus we have:
$Q=\frac{(\rho_{Oil}-\rho_{Air}) g d}{U} \frac{4\pi r^3}{3}$
But fo that formula we also need the radius of the droplet, that is beeing calculated after the following formula:
$r=\sqrt{\frac{9 \eta s}{2 g t (\rho_{Oil}-\rho_{Air})}}$

In [24]:
s=[]
for i in range(len(schweben_data["s/Skalen"])):
    s.append(ufloat(schweben_data["s/Skalen"][i], schweben_data["s_err"][i]))
s=np.array(s)
t=[]
for i in range(len(schweben_data["t/s"])):
    t.append(ufloat(schweben_data["t/s"][i], schweben_data["t_err/s"][i]))
t=np.array(s)
r=np.sqrt(9*s*viscosity_air/(2*g*t*(density_oil-density_air)))
r
#Q=(density_oil-density_air)*4/3/np.pi/r**3*g*capacitor_distance/schweben_data["U/V"]

TypeError: loop of ufunc does not support argument 0 of type AffineScalarFunc which has no callable sqrt method